<a href="https://colab.research.google.com/github/mahersalman/Introducation_To_Cloud_Computing/blob/main/HW2/HW2_Shark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ipython ipywidgets
!pip install firebase-admin
!pip install firebase


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.6 MB/s eta 0:00:00


In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from datetime import datetime, timedelta
import plotly.graph_objects as go



In [ ]:
!git clone https://github.com/mahersalman/Introducation_To_Cloud_Computing.git
json_path = './Introducation_To_Cloud_Computing/HW2/OnshapeJson0.json'
with open(json_path, "r") as fid:
    data = json.load(fid)

Cloning into 'Introducation_To_Cloud_Computing'...
remote: Enumerating objects: 97, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 97 (delta 21), reused 22 (delta 2), pack-reused 0
Receiving objects: 100% (97/97), 2.04 MiB | 14.72 MiB/s, done.
Resolving deltas: 100% (21/21), done.


In [ ]:
df = pd.DataFrame(data)
df['Time'] = pd.to_datetime(df['Time'])



## PageBase class

In [ ]:
# Base class for pages
class PageBase:
    def __init__(self):
        self.container = widgets.VBox()
        self.create_widgets()
        self.hide()  # Hide content initially

    def create_widgets(self):
        raise NotImplementedError("Subclasses should implement create_widgets()")

    def show(self):
        self.container.layout.display = 'block'

    def hide(self):
        self.container.layout.display = 'none'




# Main Page class

In [ ]:

class MainPage(PageBase):
    def __init__(self):
        super().__init__()
        self.year_dropdown = None
        self.project_dropdown = None
        self.employees = self.getEmployees()
        self.create_widgets()

    def create_widgets(self):
        self.content = widgets.HTML(value='<h1>Main Page Content</h1>')
        self.container = widgets.VBox()
        self.container.children = [self.content]

        self.basic_stats = widgets.HTML(value="")
        self.output_grid = [[widgets.Output(), widgets.Output()],
                            [widgets.Output(), widgets.Output()]]

        self.grid = widgets.GridBox(
            children=[output for row in self.output_grid for output in row],
            layout=widgets.Layout(
                grid_template_columns="repeat(2, 1fr)",
                grid_template_rows="repeat(2, auto)",
                grid_gap="20px"
            )
        )

        self.update_main_info()
        self.container.children = [self.content, self.basic_stats, self.grid]

        # Dropdown for selecting the year
        self.year_dropdown = widgets.Dropdown(
            options=df['Time'].dt.year.unique(),
            description='Year:',
            disabled=False,
        )

        # Dropdown for selecting the year for projects graph
        self.project_dropdown = widgets.Dropdown(
            options=df['Time'].dt.year.unique(),
            description='Year:',
            disabled=False,
        )

        # Function to update the plot when the dropdown value changes
        def on_year_change(change):
            with self.output_grid[1][0]:
                clear_output(wait=True)
                display(self.year_dropdown)
                self.GraphTotalAction(change['new'])

        # Function to update the projects plot when the dropdown value changes
        def on_project_year_change(change):
            with self.output_grid[1][1]:
                clear_output(wait=True)
                display(self.project_dropdown)
                self.GraphActionsPerProjects(change['new'])

        # Attach the event listener to the dropdown
        self.year_dropdown.observe(on_year_change, names='value')
        self.project_dropdown.observe(on_project_year_change, names='value')

        # Place dropdowns above respective graphs without initially plotting the graphs
        with self.output_grid[1][0]:
            clear_output(wait=True)
            display(self.year_dropdown)
            self.GraphTotalAction(2022)


        with self.output_grid[1][1]:
            clear_output(wait=True)
            display(self.project_dropdown)
            self.GraphActionsPerProjects(2022)


        with self.output_grid[0][0]:
            clear_output(wait=True)
            display(HTML(self.employees))

    def getEmployees(self):
        Employees = {}
        for emp in df['User'].unique():
            tmp = []
            tmp.append(len(df[df['User'] == emp]))
            tmp.append(len((df[df['User'] == emp])['Document'].unique()))
            Employees[emp] = tmp

        html_table = """
        <style>
            table {
                width: 100%;
                border-collapse: collapse;
                background-color: white;
            }
            th, td {
                padding: 12px;
                text-align: center;
                border-bottom: 1px solid #ddd;
                color: black;
            }
            th {
                background-color: #4CAF50;
                color: white;
            }
            tr:hover {
                background-color: #f5f5f5;
            }
            .gray-text {
                color: gray; /* Set the text color to gray */
            }
        </style>
        <table>
            <thead>
                <tr>
                    <th>Employee</th>
                    <th>Projects</th>
                </tr>
            </thead>
            <tbody>
        """

        # Add the employee names and actions
        for emp, value in Employees.items():
            html_table += f"""
                <tr>
                    <td>
                        <div>{emp}</div>
                        <div class="gray-text">{value[0]} Actions</div>
                    </td>
                    <td>
                        {value[1]}
                    </td>
                </tr>
            """

        html_table += """
            </tbody>
        </table>
        """
        return html_table

    def CalculateTotalActions(self, year):
        totalActionsPerMonths = [0] * 12
        for i in range(12):
            totalActionsPerMonths[i] = len(df[(df['Time'].dt.year == year) & (df['Time'].dt.month == (i + 1))])
        return totalActionsPerMonths

    def GraphTotalAction(self, year=None):
        if year is None:
            return
        thisYear = self.CalculateTotalActions(year)
        previousYear = self.CalculateTotalActions(year - 1)
        monthsName = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

        frame = pd.DataFrame({
            'Month': monthsName,
            str(year): thisYear,
            str(year - 1): previousYear
        })

        # Plot
        plt.figure(figsize=(12, 8))
        plt.plot(frame['Month'], frame[str(year)], label=str(year), color='blue', marker='o')
        plt.plot(frame['Month'], frame[str(year - 1)], label=str(year - 1), color='red', marker='o')
        plt.xlabel('Month')
        plt.ylabel('Total Actions')
        plt.title('Total Actions')
        plt.legend()
        plt.grid(True)
        plt.show()

    def GraphActionsPerProjects(self, year=None):
        if year is None:
            return
        projects = df['Document'].unique()
        actionsPerProject = []
        for i in projects:
            actionsPerProject.append(len(df[(df['Document'] == i) & (df['Time'].dt.year == year)]))

        frame = pd.DataFrame({
            'Document': projects,
            'Actions': actionsPerProject,
        })

        # Sort by 'Actions' and select the top 5
        frame = frame.sort_values(by='Actions', ascending=False).head(5)

        # Plot the top 5 projects
        plt.figure(figsize=(12, 8))
        plt.barh(frame['Document'], frame['Actions'], color='skyblue')
        plt.title('Top 5 Projects by Actions')
        plt.xlabel('Number of Actions')

        for index, value in enumerate(frame['Actions']):
            plt.text(value, index, str(value), va='center')

        plt.show()

    def update_main_info(self):
        totalProjects = len(df['Document'].unique())
        TotalActions = len(df)
        TotalEmployees = len(df['User'].unique())

        self.basic_stats.value = f"""
        <div style="display: flex; justify-content: center; align-items: center; gap: 10px; padding: 10px;">
            <div style="display: grid; grid-template-rows: auto auto; grid-template-columns: auto auto; align-items: center; width: 40px; height: 40px; ">
                <div style="grid-row: span 2; grid-column: 1; font-size: 30px;">📁</div>
                <div style="grid-row: 1; grid-column: 2; font-size: 18px; font-weight: bold; margin: 0; padding: 0; ">{totalProjects}</div>
                <div style="grid-row: 2; grid-column: 2; margin-left: 5px; font-size: 16px; align-self: start; font-weight: bold; margin: 0; padding: 0;">Projects</div>
            </div>
            <div style="padding:10px 60px 40px;"> </div>
            <div style="display: grid; grid-template-rows: auto auto; grid-template-columns: auto auto; align-items: center; width: 40px; height: 40px; ">
                <div style="grid-row: span 2; grid-column: 1; font-size: 30px;">⚙️</div>
                <div style="grid-row: 1; grid-column: 2; font-size: 18px; font-weight: bold; margin: 0; padding: 0; ">{TotalActions}</div>
                <div style="grid-row: 2; grid-column: 2; margin-left: 5px; font-size: 16px; align-self: start; font-weight: bold; margin: 0; padding: 0;">Actions</div>
            </div>
            <div style="padding:10px 60px 40px;"> </div>
            <div style="display: grid; grid-template-rows: auto auto; grid-template-columns: auto auto; align-items: center; width: 40px; height: 40px; ">
                <div style="grid-row: span 2; grid-column: 1; font-size: 30px;">👥</div>
                <div style="grid-row: 1; grid-column: 2; font-size: 18px; font-weight: bold; margin: 0; padding: 0; ">{TotalEmployees}</div>
                <div style="grid-row: 2; grid-column: 2; margin-left: 5px; font-size: 16px; align-self: start; font-weight: bold; margin: 0; padding: 0;">Employees</div>
            </div>
        </div>
        <p><p>

        """


# Projects class

In [ ]:
#@title helper functions

def count_actions(project_name):
    actions_count = df[df['Document'] == project_name].shape[0]
    return actions_count

def calculate_modeling_time(project_name):
    project_df = df[df['Document'] == project_name]
    start_time = project_df['Time'].min()
    end_time = project_df['Time'].max()
    modeling_time = end_time - start_time
    return modeling_time

def last_edited_date(project_name):
    project_df = df[df['Document'] == project_name]
    last_edited = project_df['Time'].max()
    return last_edited


In [ ]:
#@title Projects main code
class ProjectsPage(PageBase):
    def create_widgets(self):
        self.project_dropdown = widgets.Dropdown(
            options=df['Document'].unique(),
            description='Project:',
            disabled=False,
            layout=widgets.Layout(margin='20px 10px')
        )

        self.project_title = widgets.HTML(value="")
        #self.employee_count = widgets.HTML(value="")
        #self.actions_count = widgets.HTML(value="")
        self.basic_stats = widgets.HTML(value="")

        self.output_grid = [[widgets.Output() for _ in range(2)] for _ in range(3)]
        #self.output_grid = [[widgets.Output(layout=widgets.Layout(border='1px solid black')) for _ in range(2)] for _ in range(3)]

        self.grid = widgets.GridBox(
            children=[output for row in self.output_grid for output in row],
            layout=widgets.Layout(
                grid_template_columns="repeat(2, 1fr)",
                grid_template_rows="repeat(4, auto)",
                grid_gap="20px"
            )
        )
        self.project_dropdown.observe(self.on_project_change, names='value')
        self.container.children = [self.project_dropdown, self.project_title, self.basic_stats, self.grid]

    def on_project_change(self, change):
        project_name = change.new

        # Update project title and stats
        self.project_title.value = f"<div style=\"text-align: center; font-size: 16px; font-family: sans-serif;\"> <h2>Project: {project_name}</h2> </div>"
        self.update_project_stats(project_name)



        with self.output_grid[0][0]:
            clear_output(wait=True)
            modeling_time = calculate_modeling_time(project_name)
            last_edit = last_edited_date(project_name)
            display(widgets.HTML(f"""<div style="font-size: 16px; background-color: #f7fafc; width: 800px; height: 60px; margin: 0 auto; display: flex; padding: 20px; border-radius: 15px; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);">
              <div style="color: black;">Overall modeling time: {modeling_time}<br>
                It was last edited on: {last_edit}

              </div>
              </div>"""))

        # with self.output_grid[2][0]:
        #     clear_output(wait=True)



        with self.output_grid[1][0]:
            clear_output()
            html1=self.create_chartjs_bar_chart(project_name, "actions_per_tab")
            display(widgets.HTML(value=html1))

        with self.output_grid[1][1]:
            clear_output()
            html2=self.create_chartjs_pie_chart(project_name, "user_action_distribution")
            display(widgets.HTML(value=html2))

        with self.output_grid[2][0]:
            clear_output()
            html3=self.create_chartjs_line_chart(project_name, "modeling_time_per_date")
            display(widgets.HTML(value=html3))

        with self.output_grid[2][1]:
            clear_output()
            html4 = self.create_operations_bar_chart(project_name)
            display(widgets.HTML(value=html4))


    def update_project_stats(self, project_name):
        num_employees = df[df['Document'] == project_name]['User'].nunique()
        num_actions = df[df['Document'] == project_name].shape[0]
        self.basic_stats.value= f"""<div style="display: flex; justify-content: center; align-items: center; gap: 10px; padding: 10px;">
        <div style="display: grid; grid-template-rows: auto auto; grid-template-columns: auto auto; align-items: center; width: 40px; height: 40px; ">
          <div style="grid-row: span 2; grid-column: 1; font-size: 30px;">👥</div>
          <div style="grid-row: 1; grid-column: 2; font-size: 18px; font-weight: bold; margin: 0; padding: 0; ">{num_employees}</div>
          <div style="grid-row: 2; grid-column: 2; margin-left: 5px; font-size: 16px; align-self: start; font-weight: bold; margin: 0; padding: 0;">Employees</div>
        </div>
        <div style="padding:10px 60px 40px;"> </div>
        <div style="display: grid; grid-template-rows: auto auto; grid-template-columns: auto auto; align-items: center; width: 40px; height: 40px; ">
          <div style="grid-row: span 2; grid-column: 1; font-size: 30px;">🖥️</div>
          <div style="grid-row: 1; grid-column: 2; font-size: 18px; font-weight: bold; margin: 0; padding: 0; ">{num_actions}</div>
          <div style="grid-row: 2; grid-column: 2; margin-left: 5px; font-size: 16px; align-self: start; font-weight: bold; margin: 0; padding: 0;">Actions</div>
        </div>

</div>
"""
        #self.employee_count.value = f"Total employees working on project: {num_employees}"
        #self.actions_count.value = f"Total number of actions: {num_actions}"



    def get_operations_data(self, project_name):
      # Function to check if a description contains any of the specified keywords
      def is_undo_cancel_delete(description):
        return any(word in description.lower() for word in ["undo", "cancel", "delete"])

      def is_insert_feature(description):
        return "insert feature" in description.lower()

      def is_edit(description):
        return "edit" in description.lower()


      # Filter data for the specific project
      project_data = df[df['Document'] == project_name].copy()
      project_data['Date'] = project_data['Time'].dt.date

      # Group by date and count occurrences of each type of action
      operation_counts = project_data.groupby('Date')['Description'].apply(
          lambda x: pd.Series({
              'Delete_Cancel_Undo': x.apply(is_undo_cancel_delete).sum(),
              'Insert_Feature': x.apply(is_insert_feature).sum(),
              'Edit': x.apply(is_edit).sum()
          })
      ).reset_index()

      # Ensure all columns are present even if they have zero counts
      for op_type in ['Delete_Cancel_Undo', 'Insert_Feature', 'Edit']:
          if op_type not in operation_counts.columns:
              operation_counts[op_type] = 0

      return operation_counts



    def get_labels(self, project_name, chart_type):
        if chart_type == "actions_per_tab":
            project_data = df[df['Document'] == project_name]
            return project_data['Tab'].value_counts().index.tolist()
        elif chart_type == "user_action_distribution":
            project_data = df[df['Document'] == project_name]
            return project_data['User'].value_counts().index.tolist()
        elif chart_type == "modeling_time_per_date":
            project_data = df[df['Document'] == project_name].copy()
            project_data['Date'] = project_data['Time'].dt.date
            return [date.strftime('%Y-%m-%d') for date in project_data.groupby('Date').size().index]
        return []

    def get_data(self, project_name, chart_type):
      if chart_type == "actions_per_tab":
          project_data = df[df['Document'] == project_name]
          return project_data['Tab'].value_counts().tolist()
      elif chart_type == "user_action_distribution":
          project_data = df[df['Document'] == project_name]
          counts = project_data['User'].value_counts()
          total = counts.sum()
          return (counts / total * 100).tolist()  # Convert to percentages
      elif chart_type == "modeling_time_per_date":
          project_data = df[df['Document'] == project_name].copy()
          project_data['Date'] = project_data['Time'].dt.date
          return (project_data.groupby('Date')
                  .apply(lambda x: (x['Time'].max() - x['Time'].min()).total_seconds() / 3600)
                  .tolist())
      return []



    def create_chartjs_pie_chart(self, project_name, chart_type):
        labels = self.get_labels(project_name, chart_type)
        data = self.get_data(project_name, chart_type)
        if len(labels) != len(data):
          raise ValueError("Labels and data length mismatch.")
        html = f"""
        <div style=" background-color: #f7fafc; width: 800px; height: 400px; margin: 0 auto; display: flex; flex-direction: column; justify-content: center; align-items: center; padding: 20px; border-radius: 15px; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);">
            <div><h2>Percentage of Actions per User </h2></div>
            <canvas id="pieChart" width="400" height="200"></canvas>
            <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
            <script>
                var ctx = document.getElementById('pieChart').getContext('2d');
                var myPieChart = new Chart(ctx, {{
                    type: 'pie',
                    data: {{
                        labels: {json.dumps(labels)},
                        datasets: [{{
                            data: {json.dumps(data)},
                            backgroundColor: ['#FF6384', '#36A2EB', '#FFCE56'],
                        }}]
                    }},
                    options: {{
                        responsive: true,
                        plugins: {{
                            legend: {{
                                position: 'left',
                            }},
                            tooltip: {{
                                callbacks: {{
                                    label: function(context) {{
                                        let label = context.label || '';
                                        if (context.parsed) {{
                                            label += ': ' + context.parsed + '%';
                                        }}
                                        return label;
                                    }}
                                }}
                            }}
                        }}
                    }}
                }});
            </script>
        </div>
        """
        return html
        #display(widgets.HTML(value=html))


    def create_chartjs_bar_chart(self, project_name, chart_type):
        labels = self.get_labels(project_name, chart_type)
        data = self.get_data(project_name, chart_type)
        html = f"""
        <div style=" background-color: #f7fafc; width: 800px; height: 400px; margin: 0 auto; display: flex; flex-direction: column; justify-content: center; align-items: center; padding: 20px; border-radius: 15px; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);">
            <div><h2>Actions number per Tab</h2></div>
            <canvas id="barChart" width="400" height="200"></canvas>
            <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
            <script>
                var ctx = document.getElementById('barChart').getContext('2d');
                var myBarChart = new Chart(ctx, {{
                    type: 'bar',
                    data: {{
                        labels: {json.dumps(labels)},
                        datasets: [{{
                            label: 'Number of Actions',
                            data: {json.dumps(data)},
                            backgroundColor: 'rgba(75, 192, 192, 0.4)',
                            borderColor: 'rgba(75, 192, 192, 1)',
                            borderWidth: 1
                        }}]
                    }},
                    options: {{
                        scales: {{
                            x: {{
                                beginAtZero: true,
                                title: {{
                                  display: true,
                                  text: 'Tab Name' // Your x-axis label
                              }}
                            }},
                            y: {{
                                beginAtZero: true
                            }}
                        }},
                        responsive: true
                    }}
                }});
            </script>
        </div>
        """
        return html
        #display(widgets.HTML(value=html))


    def create_chartjs_line_chart(self, project_name, chart_type):
        labels = self.get_labels(project_name, chart_type)
        data = self.get_data(project_name, chart_type)
        html = f"""
        <div style=" background-color: #f7fafc; width: 800px; height: 400px; margin: 0 auto; display: flex; flex-direction: column; justify-content: center; align-items: center; padding: 20px; border-radius: 15px; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);">
            <div><h2>Modeling time per Date </h2></div>
            <canvas id="lineChart" width="400" height="200" ></canvas>
            <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
            <script>
                var ctx = document.getElementById('lineChart').getContext('2d');
                var myLineChart = new Chart(ctx, {{
                    type: 'line',
                    data: {{
                        labels: {json.dumps(labels)},
                        datasets: [{{
                            label: 'Modeling Time',
                            data: {json.dumps(data)},
                            fill: false,
                            borderColor: 'rgba(255, 159, 64, 1)',
                            tension: 0.1
                        }}]
                    }},
                    options: {{
                        responsive: true,
                        scales: {{
                            x: {{
                                title: {{
                                    display: true,
                                    text: 'Date'
                                }}
                            }},
                            y: {{
                                title: {{
                                    display: true,
                                    text: 'Modeling Time (Hours)'
                                }},
                                beginAtZero: true
                            }}
                        }}
                    }}
                }});
            </script>
        </div>
        """
        return html
        #display(widgets.HTML(value=html))






    def get_operations_data(self, project_name):
      # Function to check if a description contains any of the specified keywords
      def is_undo_cancel_delete(description):
        result= any(word in description.lower() for word in ["undo", "cancel", "delete"])


        return result

      def is_insert_feature(description):
        return "insert feature" in description.lower()

      def is_edit(description):
        return "edit" in description.lower()


      # Filter data for the specific project
      project_data = df[df['Document'] == project_name].copy()
      project_data['Date'] = project_data['Time'].dt.date

       # Create new columns for each action type
      project_data['Delete_Cancel_Undo'] = project_data['Description'].apply(is_undo_cancel_delete)
      project_data['Insert_Feature'] = project_data['Description'].apply(is_insert_feature)
      project_data['Edit'] = project_data['Description'].apply(is_edit)

      # Group by date and count occurrences of each type of action
      operation_counts = project_data.groupby('Date').agg({
          'Delete_Cancel_Undo': 'sum',
          'Insert_Feature': 'sum',
          'Edit': 'sum'
      }).reset_index()

      return operation_counts




    def create_operations_bar_chart(self, project_name):
        operation_counts = self.get_operations_data(project_name)

        labels = operation_counts['Date'].astype(str).tolist()
        delete_cancel_undo_data = operation_counts['Delete_Cancel_Undo'].tolist()
        insert_feature_data = operation_counts['Insert_Feature'].tolist()
        edit_data = operation_counts['Edit'].tolist()


        html = f"""
        <div style=" background-color: #f7fafc; width: 800px; height: 400px; margin: 0 auto; display: flex; flex-direction: column; justify-content: center; align-items: center; padding: 20px; border-radius: 15px; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);">
            <div><h2>Operations Count per Date</h2></div>
            <canvas id="operationsBarChart" width="400" height="200"></canvas>
            <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
            <script>
                var ctx = document.getElementById('operationsBarChart').getContext('2d');
                var myBarChart = new Chart(ctx, {{
                    type: 'bar',
                    data: {{
                        labels: {json.dumps(labels)},
                        datasets: [
                            {{
                                label: 'Delete/Cancel/Undo',
                                data: {json.dumps(delete_cancel_undo_data)},
                                backgroundColor: 'rgba(255, 99, 132, 0.2)',
                                borderColor: 'rgba(255, 99, 132, 1)',
                                borderWidth: 1
                            }},
                            {{
                                label: 'Insert Feature',
                                data: {json.dumps(insert_feature_data)},
                                backgroundColor: 'rgba(54, 162, 235, 0.2)',
                                borderColor: 'rgba(54, 162, 235, 1)',
                                borderWidth: 1
                            }},
                            {{
                                label: 'Edit',
                                data: {json.dumps(edit_data)},
                                backgroundColor: 'rgba(75, 192, 192, 0.2)',
                                borderColor: 'rgba(75, 192, 192, 1)',
                                borderWidth: 1
                            }}
                        ]
                    }},
                    options: {{
                      responsive: true,
                        scales: {{
                            x: {{
                                beginAtZero: true,
                                title: {{
                                    display: true,
                                    text: 'Date'
                                }}
                            }},
                            y: {{
                                beginAtZero: true,
                                title: {{
                                    display: true,
                                    text: 'Operation Count'
                                }}
                            }}
                        }}

                    }}
                }});
            </script>
        </div>
        """
        return html




# Employees Page class

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output

# Assuming df is your DataFrame and 'data' is your initial data
df = pd.DataFrame(data)
df['Time'] = pd.to_datetime(df['Time'])

# Define functions for additional data analysis
def countCreatForUser(user):
    filtered_df = df[df['User'] == user]
    dscrptns = filtered_df['Description']
    cnt = 0
    st = "Create"
    st2 = "created"
    for i in dscrptns:
        splited = i.split(" ")
        if splited[0] == st:
            cnt += 1
        for j in splited:
            if j == st2:
                cnt += 1
    return cnt

def countDeletsForUser(user):
    filtered_df = df[df['User'] == user]
    dscrptns = filtered_df['Description']
    cnt = 0
    st = "Delete"
    for i in dscrptns:
        splited = i.split(" ")
        if splited[0] == st:
            cnt += 1
    return cnt

def countAddOrInsertForUser(user):
    filtered_df = df[df['User'] == user]
    dscrptns = filtered_df['Description']
    cnt = 0
    for i in dscrptns:
        splited = i.split(" ")
        if splited[0] == "Add" or splited[0] == "Insert":
            cnt += 1
    return cnt

def countProjectsForUser(user):
    filtered_df = df[df['User'] == user]
    user_counts = filtered_df['Document'].value_counts()
    return user_counts.size

def calcTime(user):
    filtered_df = df[df['User'] == user]
    filtered_df = filtered_df.drop_duplicates(subset=['Time'])
    times = filtered_df['Time']
    length_of_times = len(times)
    hours = int(length_of_times // 3600)
    minutes = int(length_of_times // 60)
    seconds = int(length_of_times % 60)
    time = f"{hours}:{minutes}:{seconds}"
    return time

class EmployeesPage(PageBase):
    def create_widgets(self):
        # Dropdown for selecting year
        self.year_dropdown = widgets.Dropdown(
            options=df['Time'].dt.year.unique(),
            description='Year:',
            disabled=False,
        )

        # Dropdown for selecting month
        self.month_dropdown = widgets.Dropdown(
            options=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
            description='Month:',
            disabled=False,
        )

        # Dropdown for selecting user
        Users = df['User'].unique()
        self.user_dropdown = widgets.Dropdown(
            options=Users,
            description='User:',
            disabled=False,
        )

        # Button to trigger filtering
        self.filter_button = widgets.Button(description='Filter')

        # Output widget for displaying the result
        self.output = widgets.Output()

        # Initialize student_name with an empty string
        self.student_name = widgets.HTML(value="<h2></h2>")

        # Attach the button click event to the filtering function
        self.filter_button.on_click(self.filter_data)

        # Attach dropdown change events to update data items
        self.user_dropdown.observe(self.user_selection_change, names='value')

        # Add widgets to the container
        self.container.children = [
            widgets.HBox([self.year_dropdown, self.month_dropdown, self.user_dropdown, self.filter_button]),
            self.output,
            self.student_name
        ]

    def filter_data(self, b):
        # Get selected year, month, and user
        selected_year = self.year_dropdown.value
        selected_month = self.month_dropdown.value
        selected_user = self.user_dropdown.value

        # Filter the data based on the selected year, month, and user
        filtered_df = df[
            (df['Time'].dt.year == selected_year) &
            (df['Time'].dt.month == selected_month) &
            (df['User'] == selected_user)
        ]

        if filtered_df.empty:
            with self.output:
                clear_output()
                print(f'No data available for {selected_user} in {selected_year}-{selected_month}.')
            return

        # Count actions by type and project
        actions_by_project = filtered_df.groupby('Document').apply(lambda g: pd.Series({
            'Created': g['Description'].str.contains('Create', case=False, na=False).sum(),
            'Edited': g['Description'].str.contains('Edit', case=False, na=False).sum(),
            'Deleted': g['Description'].str.contains('Delete', case=False, na=False).sum()
        })).reset_index()

        # Plot the distribution of actions by day
        daily_actions = filtered_df.groupby(filtered_df['Time'].dt.date).size()
        fig, ax = plt.subplots(figsize=(10, 6))
        daily_actions.plot(kind='bar', ax=ax)
        ax.set_title(f'Actions Distribution by Day for {selected_user} in {selected_year}-{selected_month}')
        ax.set_xlabel('Date')
        ax.set_ylabel('Number of Actions')
        plt.xticks(rotation=45)
        plt.tight_layout()

        # Create a figure canvas for the plot
        canvas = fig.canvas

        # Display results
        with self.output:
            clear_output()
            print(f'Total actions for {selected_user} in {selected_year}-{selected_month}: {filtered_df.shape[0]}')

            # Create and display the table
            table = widgets.HTML(
                value=actions_by_project.to_html(index=False),
                layout=widgets.Layout(overflow_x='auto', width='50%')
            )

            # Display the bar chart in a widget
            plot_output = widgets.Output()
            with plot_output:
                plt.show(fig)

            # Arrange table and graph side by side
            display(widgets.HBox([table, plot_output]))

        # Update student-specific data
        self.update_student_name(selected_user)

    def user_selection_change(self, change):
        # Triggered when the user selection changes
        selected_user = change.new
        self.update_student_name(selected_user)

    def update_student_name(self, student_name):
        self.student_name.value = f"<h2 style='text-align:center; margin:10px 0; color: #333;'>{student_name}</h2>"
        self.update_data_items(student_name)

    def update_data_items(self, student_name):
        self.data_items = [
            (countProjectsForUser(student_name), "Projects"),
            (calcTime(student_name), "Modeling Time"),
            (countCreatForUser(student_name), "Elements Created"),
            (countAddOrInsertForUser(student_name), "Elements Added/Inserted"),
            (countDeletsForUser(student_name), "Elements Deleted")
        ]

        # Create data boxes
        self.data_boxes = []
        for value, label in self.data_items:
            box = widgets.HTML(f"""
                <div style="
                    background-color: #f8f9fa;
                    border: 1px solid #e9ecef;
                    border-radius: 5px;
                    padding: 15px;
                    text-align: center;
                    height: 100%;
                    box-shadow: 0 2px 5px rgba(0,0,0,0.1);
                ">
                    <h3 style="margin:5px; font-size: 24px; color: #333;">{value}</h3>
                    <p style="margin:5px; color: #6c757d;">{label}</p>
                </div>
            """)
            self.data_boxes.append(box)

        # Update the layout with data boxes
        self.update_data_boxes()

    def update_data_boxes(self):
        # Add data boxes to the layout
        data_boxes_layout = widgets.GridBox(
            self.data_boxes,
            layout=widgets.Layout(
                grid_template_columns="repeat(5, 1fr)",
                grid_template_areas="""
                    "data1 data2 data3 data4 data5"
                """,
                width='100%',
                grid_gap='15px',
                padding='20px'
            )
        )
        self.container.children = [
            widgets.HBox([self.year_dropdown, self.month_dropdown, self.user_dropdown, self.filter_button]),
            self.output,
            self.student_name,
            data_boxes_layout
        ]


# Notifications Page class

In [ ]:

class NotificationsPage(PageBase):
    def create_widgets(self):
        self.content = widgets.HTML(value='<h1>Notifications Page Content</h1>')
        self.container.children = [self.content]
        self.data = data
        # Convert string time to datetime object
        for entry in self.data:
          if isinstance(entry['Time'], str):
            entry['Time'] = datetime.strptime(entry['Time'], '%Y-%m-%d %H:%M:%S')

        # Sort data by time (oldest to newest)
        self.data.sort(key=lambda x: x['Time'])

        # Sort data by time (oldest to newest)
        self.data.sort(key=lambda x: x['Time'])


        # Helper function to detect undo/cancel/delete actions
        def is_undo_cancel_delete(description):
            return any(word in description.lower() for word in ["undo", "cancel", "delete"])

        # Function to detect Case 1
        def detect_case_1(data):
            notifications = []
            notified_combinations = set()
            for i, entry in enumerate(data):
                for j in range(i + 1, len(data)):
                    if (data[j]['Time'] - entry['Time']) > timedelta(minutes=5):
                        break
                    if entry['Document'] == data[j]['Document'] and entry['Tab'] == data[j]['Tab'] and entry['User'] != data[j]['User']:
                        combination = (entry['Document'], entry['Tab'], entry['User'], data[j]['User'])
                        if combination not in notified_combinations and combination[::-1] not in notified_combinations:
                            notification = {
                                "Time": data[j]['Time'],
                                "Title": "Heightened conflict chance.",
                                "Body": f"Your employees {entry['User']} & {data[j]['User']} are working simultaneously on Project \"{entry['Document']}\", Tab \"{entry['Tab']}\"."
                            }
                            notifications.append(notification)
                            notified_combinations.add(combination)
            return notifications

        # Function to detect Case 2
        def detect_case_2(data):
            notifications = []
            action_count = {}
            for entry in data:
                if is_undo_cancel_delete(entry['Description']):
                    user = entry['User']
                    if user not in action_count:
                        action_count[user] = []
                    action_count[user].append(entry['Time'])

                    # Remove actions older than 5 minutes
                    action_count[user] = [time for time in action_count[user] if entry['Time'] - time <= timedelta(minutes=5)]

                    if len(action_count[user]) > 5:
                        notification = {
                            "Time": entry['Time'],
                            "Title": "Your employee might need help.",
                            "Body": f"Your employee {user} is backing from a number of operations. You might want to consider setting up another team member for help."
                        }
                        notifications.append(notification)
                        action_count[user] = []
            return notifications

        # Function to detect Case 3
        def detect_case_3(data):
            notifications = []
            last_worked = {}
            current_time = datetime.now()
            cutoff_time = current_time - timedelta(weeks=2)

            for entry in data:
                project = entry['Document']
                if project not in last_worked or entry['Time'] > last_worked[project]:
                    last_worked[project] = entry['Time']

            for project, last_time in last_worked.items():
                if last_time < cutoff_time:
                    notification = {
                        "Time": last_time,
                        "Title": "Ditched Project",
                        "Body": f"\"{project}\" hasn’t had any work done for more than two weeks. Consider taking a look."
                    }
                    notifications.append(notification)
            return notifications

        # Function to detect Case 4
        def detect_case_4(data):
            notifications = []
            project_actions = {}
            for entry in data:
                if is_undo_cancel_delete(entry['Description']):
                    project = entry['Document']
                    if project not in project_actions:
                        project_actions[project] = []
                    project_actions[project].append(entry['Time'])

                    # Remove actions older than 5 minutes
                    project_actions[project] = [time for time in project_actions[project] if entry['Time'] - time <= timedelta(minutes=5)]

                    if len(project_actions[project]) > 5:
                        notification = {
                            "Time": entry['Time'],
                            "Title": "Your project might be facing difficulties.",
                            "Body": f"\"{project}\" has had multiple undo/cancel/delete operations recently. Consider taking a look."
                        }
                        notifications.append(notification)
                        project_actions[project] = []
            return notifications

        def filter_notifications(notifications):
            filtered_notifications = []
            seen_bodies = {}

            for notification in notifications:
                body = notification['Body']
                time = notification['Time']

                if body not in seen_bodies:
                    seen_bodies[body] = time
                    filtered_notifications.append(notification)
                else:
                    last_time = seen_bodies[body]
                    if time - last_time > timedelta(hours=1):
                        seen_bodies[body] = time
                        filtered_notifications.append(notification)

            return filtered_notifications

        # Combine notifications from all cases and order by date
        def combine_notifications(data):
            notifications = []
            notifications.extend(detect_case_1(data))
            notifications.extend(detect_case_2(data))
            notifications.extend(detect_case_3(data))
            notifications.extend(detect_case_4(data))

            # Sort notifications by time
            notifications.sort(key=lambda x: x['Time'], reverse=True)

            # Filter notifications to keep only one within an hour for the same body
            filtered_notifications = filter_notifications(notifications)

            return filtered_notifications

        # Get combined and filtered notifications
        notifications = combine_notifications(self.data)


        # Create notification widgets and add to the container
        notification_widgets = []
        for notification in notifications:
            notification_html = widgets.HTML(
                value=f"""
                <div style="border: 1px solid #ccc; padding: 15px; margin: 10px 0; background-color: #f4f4f4; border-radius: 8px; display: flex; align-items: center; justify-content: space-between;">
                  <div style="flex: 4; padding-left: 20px;">
                      <strong style="font-size: 1.2em; display: block; margin-bottom: 5px; color: #333;">
                          {notification['Title']}
                      </strong>
                      <p style="margin: 0; color: #555;">{notification['Body']}</p>
                  </div>
                  <div style="font-size: 0.8em; color: #666; min-width: 100px; text-align: left; align-self: flex-start;">
                      {notification['Time'].strftime('%Y-%m-%d %H:%M:%S')}
                  </div>
              </div>
                """,
                layout=widgets.Layout(width='100%')
            )
            notification_widgets.append(notification_html)

        # Update the container with the new notification widgets
        self.container.children = [self.content] + notification_widgets

# UploadJson Page class




In [ ]:

class UploadJsonPage(PageBase):
    def create_widgets(self):
        self.content = widgets.HTML(value='<h1>UploadJson Page Content</h1>')

        # Create the file upload widget
        upload_widget = widgets.FileUpload(
            accept='.json',  # Accept only .json files
            multiple=False   # Single file upload
        )

        # Create an output widget to display messages
        output = widgets.Output()
        # Attach the upload handler to the widget
        upload_widget.observe(self.on_upload_change, names='value')

        self.container.children = [self.content,upload_widget,output]

    def file_exists_in_firebase(self, file_name):
        # Check if the file already exists in Firebase
        existing_files = FBconn.get('/JsonFiles/', None)
        if existing_files:
            for key, value in existing_files.items():
                if 'file_name' in value and value['file_name'] == file_name:
                    return True, key
        return False , None

    def upload_json_to_firebase(self, file_name, file_content):
        data = json.loads(file_content.decode('utf-8'))
        # If data is a list, we can wrap it into a dictionary with the file name as one of the fields
        if isinstance(data, list):
            data = {
                "file_name": file_name,
                "data": data
            }
        result = FBconn.post('/JsonFiles/', data)
        return result

    def update_json_in_firebase(self, file_name, file_content,update_key):
        try:
            data = json.loads(file_content.decode('utf-8'))
            if isinstance(data, list):
                data = {
                    "file_name": file_name,
                    "data": data
                }

            # Update existing entry
            result = FBconn.put(f'/JsonFiles/{update_key}', name="data", data=data["data"])
            result = FBconn.put(f'/JsonFiles/{update_key}', name="file_name", data=file_name)

            return result
        except Exception as e:
            alert(f"An error occurred: {e}")

    def on_upload_change(self, change):
        output.clear_output()
        with output:
            # Check if a file has been uploaded
            if len(upload_widget.value) > 0:
                # Get the uploaded file
                file_info = list(upload_widget.value.values())[0]
                file_name = file_info['metadata']['name']
                file_content = file_info['content']

                # Check if the uploaded file is a JSON
                try:
                    json_data = json.loads(file_content.decode('utf-8'))
                    print(f"{file_name} is a valid JSON file.")
                    exist, updatekey = self.file_exists_in_firebase(file_name)
                    if exist:
                        # Create a button widget for the popup
                        update_button = widgets.Button(description="Update")
                        cancel_button = widgets.Button(description="Cancel")

                        def on_update_click(b):
                            result = self.update_json_in_firebase(file_name, file_content, updatekey)
                            print("File successfully updated in Firebase.")
                            print(result)
                            popup.close()

                        def on_cancel_click(b):
                            print("Upload cancelled.")
                            popup.close()

                        update_button.on_click(on_update_click)
                        cancel_button.on_click(on_cancel_click)

                        # Create the popup widget
                        popup = widgets.VBox([
                            widgets.HTML(f"<b>{file_name}</b> already exists. Do you want to update it?"),
                            widgets.HBox([update_button, cancel_button])
                        ])

                        display(popup)
                    else:
                        result = self.upload_json_to_firebase(file_name, file_content)
                        print("File successfully uploaded to Firebase.")
                        print(result)
                except json.JSONDecodeError:
                    print(f"{file_name} is not a valid JSON file.")


# **Main App Wrapper**

In [ ]:
df = pd.DataFrame(data)
df['Time'] = pd.to_datetime(df['Time'])
df['Document'] = df['Document'].fillna('Unknown')


# ---------------------------------------- Main Wrapper ------------------------------------------

import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output


class DashboardWidget:
    def __init__(self):
        self.create_widgets()
        self.layout_widgets()
        self.setup_navigation()
        # Custom CSS
        custom_css = """
        <style>
            .custom-toggle-buttons .widget-toggle-button {
                background-color: #f7fafc !important;
                font-size: 1rem !important;  /* Adjust font size as needed */
                padding: 0.5rem 1rem 1rem 1rem;  /* Top, Right, Bottom, Left padding */
                line-height: 1;  /* Adjust line height for better text alignment */
                height: 2rem;  /* Adjust height as needed */
            }
        </style>
        """

        # Inject custom CSS into the notebook
        display(widgets.HTML(custom_css))

    def create_widgets(self):


        # Create ToggleButtons with custom class
        self.nav_items = widgets.ToggleButtons(
            options=['Main', 'Projects', 'Employees', 'Notifications','Upload json'],
            description='',
            button_style='',
            layout=widgets.Layout(width='auto', height='auto'),
        )
        self.nav_items.add_class('custom-toggle-buttons')

        # # Icons (simplified)
        # self.icons = widgets.HBox([
        #     widgets.HTML(value='<b>Hello, Adam</b>', layout=widgets.Layout(margin='0 10px 0 900px')),
        #     widgets.Button(description='👤', layout=widgets.Layout(width='60px'))
        # ])

        # Create page instances
        self.main_page = MainPage()
        self.projects_page = ProjectsPage()
        self.employees_page = EmployeesPage()
        self.notifications_page = NotificationsPage()
        self.uploadjson_page = UploadJsonPage()

    # Function to create the header with Tailwind CSS
    def create_header(self):
      header_label = widgets.HTML("""

        <div style="display: flex; background-color: #f7fafc; padding: 1rem;">
           <div style=" display: flex; align-items: center; flex: 1;">
            <img src="https://play-lh.googleusercontent.com/yAS9WJJnjlCx77RxIvJSssrixhCdUxnBlM3CuPnQpl8QI3Ez19KreBL4xREc1gtmK_Y=w240-h480" style="height: 3rem; width: 3rem;">
            <h1 style="color:black;font-size: 1.5rem; font-family: sans-serif; font-weight: bold; margin-top: 0.5rem; margin-left: 0.5rem;">OnShape Insights</h1>
            </div>
            <div style="color:black;display: flex; align-items: center; justify-content: flex-end; flex: 1;">
              <b>Hello, Adam 👤</b></div>
        </div>
        """)
      return header_label

    def layout_widgets(self):
        top_bar = widgets.HBox([self.nav_items])

        main_content = widgets.VBox([
            self.main_page.container,
            self.projects_page.container,
            self.employees_page.container,
            self.notifications_page.container,
            self.uploadjson_page.container
        ])

        self.main_layout = widgets.VBox([self.create_header(),top_bar, main_content])
        self.main_layout.layout.width = '100%'
        self.main_layout.layout.border = '1px solid #ddd'
        self.main_layout.layout.padding = '20px'

    def setup_navigation(self):
        self.nav_items.observe(self.on_nav_change, names='value')

    def on_nav_change(self, change):
        selected_page = change.new
        if selected_page == 'Main':
            self.show_page(self.main_page)
        elif selected_page == 'Projects':
            self.show_page(self.projects_page)
        elif selected_page == 'Employees':
            self.show_page(self.employees_page)
        elif selected_page == 'Notifications':
            self.show_page(self.notifications_page)
        elif  selected_page == 'Upload json':
            self.show_page(self.uploadjson_page)

    def show_page(self, page):
        # Hide all pages
        for p in [self.main_page, self.projects_page, self.employees_page, self.notifications_page,self.uploadjson_page]:

            p.hide()

        # Show selected page
        page.show()

    def display(self):
        display(self.main_layout)




# Create and display the dashboard widget
dashboard = DashboardWidget()
dashboard.display()


HTML(value='\n        <style>\n            .custom-toggle-buttons .widget-toggle-button {\n                bac…

# Establish Firebase connection

In [ ]:
from firebase import firebase
# Firebase URL
firebase_url = 'https://cloud-tut6-2a99a-default-rtdb.firebaseio.com/'
FBconn = firebase.FirebaseApplication(firebase_url, None)

# Indexing Glossary page

https://colab.research.google.com/drive/13eGx3LSkIHQfEiNEbZqlBOC6o9ACSSMv

# Upload a json file to database

In [ ]:
# Import libraries

import json
import ipywidgets as widgets
from IPython.display import display



# Create the file upload widget
upload_widget = widgets.FileUpload(
    accept='.json',  # Accept only .json files
    multiple=False   # Single file upload
)

# Create an output widget to display messages
output = widgets.Output()

def file_exists_in_firebase(file_name):
    # Check if the file already exists in Firebase
    existing_files = FBconn.get('/JsonFiles/', None)
    if existing_files:
        for key, value in existing_files.items():
            if 'file_name' in value and value['file_name'] == file_name:
                return True, key
    return False , None

def upload_json_to_firebase(file_name, file_content):
    data = json.loads(file_content.decode('utf-8'))
    # If data is a list, we can wrap it into a dictionary with the file name as one of the fields
    if isinstance(data, list):
        data = {
            "file_name": file_name,
            "data": data
        }
    result = FBconn.post('/JsonFiles/', data)
    return result

def update_json_in_firebase(file_name, file_content,update_key):
    try:
        data = json.loads(file_content.decode('utf-8'))
        if isinstance(data, list):
            data = {
                "file_name": file_name,
                "data": data
            }

        # Update existing entry
        result = FBconn.put(f'/JsonFiles/{update_key}', name="data", data=data["data"])
        result = FBconn.put(f'/JsonFiles/{update_key}', name="file_name", data=file_name)

        return result
    except Exception as e:
        alert(f"An error occurred: {e}")

def on_upload_change(change):
    output.clear_output()
    with output:
        # Check if a file has been uploaded
        if len(upload_widget.value) > 0:
            # Get the uploaded file
            file_info = list(upload_widget.value.values())[0]
            file_name = file_info['metadata']['name']
            file_content = file_info['content']

            # Check if the uploaded file is a JSON
            try:
                json_data = json.loads(file_content.decode('utf-8'))
                print(f"{file_name} is a valid JSON file.")
                exist, updatekey = file_exists_in_firebase(file_name)
                if exist:
                    # Create a button widget for the popup
                    update_button = widgets.Button(description="Update")
                    cancel_button = widgets.Button(description="Cancel")

                    def on_update_click(b):
                        result = update_json_in_firebase(file_name, file_content, updatekey)
                        print("File successfully updated in Firebase.")
                        print(result)
                        popup.close()

                    def on_cancel_click(b):
                        print("Upload cancelled.")
                        popup.close()

                    update_button.on_click(on_update_click)
                    cancel_button.on_click(on_cancel_click)

                    # Create the popup widget
                    popup = widgets.VBox([
                        widgets.HTML(f"<b>{file_name}</b> already exists. Do you want to update it?"),
                        widgets.HBox([update_button, cancel_button])
                    ])

                    display(popup)
                else:
                    result = upload_json_to_firebase(file_name, file_content)
                    print("File successfully uploaded to Firebase.")
                    print(result)
            except json.JSONDecodeError:
                print(f"{file_name} is not a valid JSON file.")

# Attach the upload handler to the widget
upload_widget.observe(on_upload_change, names='value')

# Display the upload widget and output widget
display(upload_widget, output)

FileUpload(value={}, accept='.json', description='Upload')

Output()

# Retrieve a json file from firebase

# Delete a json file from firebase **(JsonFiles Directory)**



In [ ]:
from firebase import firebase

def delete_file_from_firebase(file_name):
    # Get the list of files
    existing_files = FBconn.get('/JsonFiles/', None)

    # Check if the file exists
    if existing_files:
        for key, value in existing_files.items():
            if 'file_name' in value and value['file_name'] == file_name:
                # Delete the file
                FBconn.delete('/JsonFiles/' + key, None)
                print(f"File '{file_name}' deleted successfully.")
                return
    print(f"File '{file_name}' not found.")


In [ ]:
#delete_file_from_firebase("OnshapeJson0.json")